In [1]:
import os

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [3]:
paths = {
    'SCRIPTS_PATH': os.path.join('scripts'),
    'APIMODEL_PATH': os.path.join('models', 'API'),
    'ANNOTATION_PATH': os.path.join('annotations'),
    'MODEL_PATH': os.path.join('models'),
    'PRETRAINED_MODEL_PATH': os.path.join('models','pretrained'),
    'CHECKPOINT_PATH': os.path.join('models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('protoc')
 }

files = {
    'PIPELINE_CONFIG':os.path.join('models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [4]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [5]:
if os.name == 'posix':  
    !cd models/API/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

Processing /media/mausneg/ntfs_disk/Surya/Language Programming/Python/advanced-computer-vision/object-detection/hand-sign/models/API/research
  Preparing metadata (setup.py) ... done
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1655476 sha256=b62b231108e56b91ea713d0bc9c6e7d10bb4698c17bca5f33c47ceb3ef927c12
  Stored in directory: /tmp/pip-ephem-wheel-cache-1_6w_s1r/wheels/e7/41/2c/ff38725eb1651baec2366fdac20ccbfaa37cdad3a031d3e668
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [6]:
import object_detection

In [8]:
labels = [
    {'name':'one', 'id':1},
    {'name':'two', 'id':2},
    {'name':'three', 'id':3},
    {'name':'four', 'id':4},
    {'name':'five', 'id':5},
    {'name':'thumbs_up', 'id':6},
    {'name':'thumbs_down', 'id':7},

]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')